# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40527,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:46423,Total threads: 4
Dashboard: http://127.0.0.1:38683/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:37197,


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,1100,Hannah,-0.745732,-0.933717
1,982,Patricia,0.712973,-0.447055
2,1051,George,-0.867196,-0.253790
3,1016,Alice,-0.154368,0.596941
4,961,George,-0.312008,0.592546


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999905e+02,-5.120842e-04,-4.561504e-04
std,3.162310e+01,5.773489e-01,5.776176e-01
min,8.430000e+02,-9.999998e-01,-9.999997e-01
25%,9.790000e+02,-4.909420e-01,-4.956294e-01
50%,1.000000e+03,6.822933e-03,5.649285e-03
75%,1.022000e+03,5.051257e-01,5.057536e-01
max,1.173000e+03,9.999979e-01,9.999995e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1016,Tim,0.8389750116340078
958,Victor,-0.4222929388166208
1021,Victor,-0.8051297833371027
1050,Ingrid,-0.6200086282657626
968,Jerry,-0.5943029977817205
1010,Oliver,-0.059642928964112
1011,Victor,-0.5876068716080096
1040,Edith,0.2540897873591941
1030,Alice,0.4607720559115991


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,1100,Hannah,-0.745732,-0.933717
1,982,Patricia,0.712973,-0.447055
2,1051,George,-0.867196,-0.253790
3,1016,Alice,-0.154368,0.596941
4,961,George,-0.312008,0.592546


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,1100,Hannah,-0.745732,-0.933717
1,982,Patricia,0.712973,-0.447055
2,1051,George,-0.867196,-0.253790
3,1016,Alice,-0.154368,0.596941
4,961,George,-0.312008,0.592546


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,1016,Tim,0.838975,NaN
1,958,Victor,-0.422293,NaN
2,1021,Victor,-0.805130,NaN
3,1050,Ingrid,-0.620009,NaN
4,968,Jerry,-0.594303,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-9054ae93-fd13-47ba-9ff2-fc2815d5b2d8')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,1100,Hannah,-0.745732,-0.933717
1,982,Patricia,0.712973,-0.447055
2,1051,George,-0.867196,-0.253790
3,1016,Alice,-0.154368,0.596941
4,961,George,-0.312008,0.592546
...,...,...,...,...
86395,1014,Kevin,-0.378957,-0.764232
86396,1023,Ray,0.812062,0.025841
86397,1022,Patricia,0.164585,-0.275643
86398,1012,Yvonne,0.215770,0.342972


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-20.csv',
 'data/2000-01-02.csv',
 'data/2000-01-01.csv',
 'data/2000-01-17.csv',
 'data/2000-01-28.csv',
 'data/2000-01-05.csv',
 'data/2000-01-26.csv',
 'data/2000-01-21.csv',
 'data/2000-01-24.csv',
 'data/2000-01-04.csv',
 'data/2000-01-06.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-03.csv',
 'data/2000-01-11.csv',
 'data/2000-01-27.csv',
 'data/2000-01-12.csv',
 'data/2000-01-09.csv',
 'data/2000-01-23.csv',
 'data/2000-01-16.csv',
 'data/2000-01-15.csv',
 'data/2000-01-08.csv',
 'data/2000-01-29.csv',
 'data/2000-01-19.csv',
 'data/2000-01-18.csv',
 'data/2000-01-30.csv',
 'data/2000-01-10.csv',
 'data/2000-01-25.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-11f14358-6e26-4371-8d9a-4acbcbfdbadd'),
 Delayed('read_data-049bdc91-e9ed-439f-9abd-949c3fd87c07'),
 Delayed('read_data-338fb6ae-2e14-4775-b326-c6bb13491621'),
 Delayed('read_data-372d5eb4-fe10-4cd8-bc29-265e79f41bd0'),
 Delayed('read_data-95a88581-6cff-4d6b-8073-0f2ea7f679cd'),
 Delayed('read_data-a5703607-3354-4a2a-93ff-433568f84559'),
 Delayed('read_data-0c4d12ec-5061-4570-bfa0-6132c4b72958'),
 Delayed('read_data-08193619-5686-40f1-a1fa-c51c7d397676'),
 Delayed('read_data-024c1829-9b83-4668-b162-db7baa2f0a3c'),
 Delayed('read_data-0fd011b1-ea7b-411f-861f-dc4922de95dd'),
 Delayed('read_data-686ff013-621d-4502-bbfd-130e0b61a7c7'),
 Delayed('read_data-70ef4eb8-4a1d-4839-8d26-50e2cc249c58'),
 Delayed('read_data-845b39a4-43d0-473f-a1de-7b9ef7018829'),
 Delayed('read_data-37857beb-ccb4-4b06-8e17-0f63f42e34c7'),
 Delayed('read_data-70647c1f-5f34-4ac1-b66d-afafcd524533'),
 Delayed('read_data-ac79f044-0506-4635-9c02-172c08781f1f'),
 Delayed('read_data-65d209c3-558a-42e1-a

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1042,Edith,0.589682,0.530279
1,940,Michael,0.922513,-0.679396
2,984,Ray,0.113801,-0.720699
3,998,Sarah,0.428700,0.953636
4,959,Jerry,-0.453112,0.729066


In [26]:
df.describe().compute()

,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999945e+02,-5.762694e-04,-4.336835e-04
std,3.162630e+01,5.773179e-01,5.776784e-01
min,8.430000e+02,-9.999998e-01,-9.999997e-01
25%,9.790000e+02,-4.909420e-01,-4.956294e-01
50%,1.000000e+03,6.822933e-03,5.649285e-03
75%,1.022000e+03,5.051257e-01,5.057536e-01
max,1.173000e+03,9.999979e-01,9.999995e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.